In [1]:
# Imports
import torch
import os 
import sys
from dask_jobqueue import SLURMCluster
from distributed import Client
from pathlib import Path

import do_simulation_causalklgp as file

In [2]:
# Args setup
ntrial = 50
n = 100
n_int = 100
niter = 1000
front_door = False
minimise = True
calibrate = False

In [3]:
# Cluster creation
cluster = SLURMCluster(
    n_workers=0,
    memory="32GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":11111",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="3:0:0",
    
    job_extra_directives = ["-p medium,fast,cpu"],
)
cluster.adapt(minimum=0, maximum=200)
client = Client(cluster)

In [4]:
# Submitting jobs
futures = []
for seed in range(ntrial):
        f = client.submit(file.main,
                              seed,
                              n,n_int,
                              niter = niter,
                              front_door = front_door,
                              minimise = minimise,
                              calibrate = calibrate
                             )
        futures += [f]

In [5]:
futures

[<Future: pending, key: main-b6205cc7ebcd5124986cade05d607a33>,
 <Future: pending, key: main-75988b9b8ec5de585007a374c9eb5dfe>,
 <Future: pending, key: main-75f7e99692588296bf411e03108477f6>,
 <Future: pending, key: main-4ffc7c690ed455f9ebf034c8cb2dc43d>,
 <Future: pending, key: main-bd9e6559047897a58ae34f5dbfbf986e>,
 <Future: pending, key: main-40dad065979ba524faadb1095fdbf01b>,
 <Future: pending, key: main-e1ad4e2bb57001f00d7fe62fe5e07802>,
 <Future: pending, key: main-c419cf5dfc4655ded8cb42a727660462>,
 <Future: pending, key: main-287f0fbb7c4e322604ad45f4daa41087>,
 <Future: pending, key: main-f9f4018f888f4436970f62ea83ecb4ab>,
 <Future: pending, key: main-3183d6458221c21230267b789e876d6f>,
 <Future: pending, key: main-40d4084e11ff52126954cf0e7907ed34>,
 <Future: pending, key: main-2279d398e480754ecff66523cbb039ac>,
 <Future: pending, key: main-6aec3ee972057158ccb73a4f68356a54>,
 <Future: pending, key: main-6ee58111f61c81e1174031631945d097>,
 <Future: pending, key: main-0db9702f5e7

In [6]:
for i in range(len(futures)):
    if futures[i].status == 'error':
        futures[i] = futures[i-1]

In [8]:
# Getting results
results = client.gather(futures)

In [9]:
# Closing client
client.close()
cluster.close()

In [10]:
# Saving results
torch.save(obj = results,
           f = "simulation_causalklgp_ntrial={0}_n={1}_frontdoor={2}_minimise={3}.pt".format(ntrial,
                                                                                             n,
                                                                                             front_door,
                                                                                             minimise)
          )